In [ ]:
import librosa
import soundfile
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [21]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        result = np.array([])

        if chroma:
            stft = np.abs(librosa.stft(X))
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma_feat = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma_feat))
        if mel:
            mel_spec = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel_spec))

    return result



In [22]:
emotions={
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}

observed_emotions=['calm', 'happy', 'fearful', 'disgust']


In [ ]:
def load_data(test_size=0.2):
    x, y = [], []
    print("🔎 Looking for .wav files...")
    for file in glob.glob("/Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_*/**/*.wav", recursive=True):
        print(f"📄 Found: {file}")
        file_name = os.path.basename(file)
        try:
            emotion = emotions[file_name.split("-")[2]]
        except KeyError:
            print(f"⚠️ Unknown emotion code in file: {file_name}")
            continue
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
        print(f"✅ Loaded: {file} → {emotion}")

    print(f"✅ Total usable files: {len(x)}")

    if len(x) == 0:
        print("❌ No valid .wav files found. Double-check folder path, file structure, and emotions.")
        return [], [], [], []

    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)



In [24]:
import glob
files = glob.glob("/Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_*/*.wav")
print(f"Total .wav files found: {len(files)}")
for f in files[:5]:
    print(f)


Total .wav files found: 1440
/Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-05-01-02-01-16.wav
/Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-06-01-02-02-16.wav
/Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-06-02-01-02-16.wav
/Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-05-02-01-01-16.wav
/Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-07-01-01-01-16.wav


In [25]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

🔎 Looking for .wav files...
📄 Found: /Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-05-01-02-01-16.wav
📄 Found: /Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-06-01-02-02-16.wav
✅ Loaded: /Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-06-01-02-02-16.wav → fearful
📄 Found: /Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-06-02-01-02-16.wav
✅ Loaded: /Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-06-02-01-02-16.wav → fearful
📄 Found: /Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-05-02-01-01-16.wav
📄 Found: /Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-07-01-01-01-16.wav
✅ Loaded: /Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-07-01-01-01-16.wav → disgust
📄 Found: /Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-04-01-01-02-16.wav
📄 Found: /Users/gracekaur/Downloads/DataFlair/ravdessdata/Actor_16/03-01-04-02-02-02-16.wav
📄 Found: /Users/gra

In [26]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

(576, 192)
Features extracted: 180


In [27]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)


In [28]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [29]:
y_pred=model.predict(x_test)

In [30]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 72.40%
